In [ ]:
# Import libraries 
import extract_eye_features
import dill
import scipy
import os

# Import the path to Pi_util
import sys 
sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import importlib

In [ ]:
# Generate a playable video
path_to_recording_chunks: str = "/Volumes/T7 Shield/FLIC_2004_gazeCalibration_tf_session1"
output_dir: str = "./STORAGE_FOLDER/VIDEO_FOLDER"
Pi_util.generate_playable_videos(path_to_recording_chunks, output_dir, apply_digital_gain=True, fill_missing_frames=True, debayer_images=True, pupil_image_rotation_correction=True)

In [ ]:
# Attempt to extract pupil + eyelid features from this video 
importlib.reload(extract_eye_features)
path_to_video: str = "./FLIC_200X_gazeCalibration_session1/P.avi"
eye_features, perimeter_dict = extract_eye_features.extract_eye_features(path_to_video, 
                                                                         is_grayscale=True, 
                                                                         visualize_results=True, 
                                                                         method='pylids',
                                                                         safe_execution=True,
                                                                         keypoint_threshold=-1
                                                                        )

In [ ]:
# Attempt to extract only pupil features from this video 
importlib.reload(extract_eye_features)
path_to_video: str = "/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2005/walkIndoor/temporalFrequency/P.avi"
pupil_features, perimeter_dict = extract_eye_features.extract_pupil_features(path_to_video, 
                                                                             is_grayscale=True, 
                                                                             visualize_results=True, 
                                                                             method='pylids',
                                                                             safe_execution=True,
                                                                             keypoint_threshold=-1,
                                                                             visualization_output_filepath="/Users/zacharykelly/Desktop/FLIC_2005_walkIndoor_tf_visualizedPupilFeatures.avi"
                                                                            )

In [23]:
# Define the path to the external SSD that contains the videos 
SSD_path: str = "/Volumes/T7 Shield/scriptedIndoorOutdoorVideos"
output_path: str = "/Users/zacharykelly/Desktop/high_contrast_perimeter_files"

# Iterate over the subjects that we have 
for subject_id in os.listdir(SSD_path):
    # Skip hidden files 
    if(subject_id.startswith(".")):
        continue 
    
    # Otherwise, construct the path to the subjecty 
    subject_path: str = os.path.join(SSD_path, subject_id)

    # Iterate over the activites of the subject 
    for activitiy in os.listdir(subject_path):
        # Skip hidden files 
        if(activitiy.startswith(".")):
            continue 
        
        # If the activity is not gazecal or walkindoor just skip 
        if(not ("gaze" in activitiy or "walkIndoor" in activitiy)):
            continue 

        # Construct the path to the activity 
        activitiy_path: str = os.path.join(subject_path, activitiy)

        # Iterate over the modes at this activity 
        for mode in os.listdir(activitiy_path):
            # Skip hidden files 
            if(mode.startswith(".")):
                continue 
            
            # Skip spatial resolution 
            if("spatial" in mode):
                continue 
            
            # construct the path to the mode directory 
            mode_path: str = os.path.join(activitiy_path, mode)

            # Find the P contrast videos for this mode 
            P_contrast_videos: list[str] = [ os.path.join(mode_path, file) for file in os.listdir(mode_path)
                                             if "P" in file and "contrast" in file
                                             and not file.startswith(".")
                                           ]
            
            # If no contrast, just skip with a message 
            if(len(P_contrast_videos) == 0):
                print(f"{mode_path} has no contrast videos")
                continue 

            # Otherwise, we are going to generate the perimeter file for this 
            print(P_contrast_videos)
            for contrast_video_path in P_contrast_videos:
                # Find the filename of this file 
                filename: str = os.path.splitext(os.path.basename(contrast_video_path))[0]

                # Construct the path to the visualization output 
                perimeter_file_output_path: str = os.path.join(output_path, subject_id, activitiy, mode, f"{filename}_perimeter.mat")
                visualization_output_filepath: str = os.path.join(output_path, subject_id, activitiy, mode, f"{filename}_visualizedPupilFeatures.avi")
                os.makedirs(visualization_output_filepath, exist_ok=True)

                print(f"Generating: {perimeter_file_output_path}")

                # Generate the perimeter file 
                pupil_features, perimeter_dict = extract_eye_features.extract_pupil_features(path_to_video, 
                                                                                             is_grayscale=True, 
                                                                                             visualize_results=True, 
                                                                                             method='pylids',
                                                                                             safe_execution=True,
                                                                                             keypoint_threshold=-1,
                                                                                             visualization_output_filepath=visualization_output_filepath
                                                                                            )
        
                # Output the perimeter file as a .mat
                scipy.io.savemat(perimeter_file_output_path, {"perimeter": perimeter_dict})


['/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2001/walkIndoor/temporalFrequency/P_contrast-1x5.avi']
Generating: /Users/zacharykelly/Desktop/high_contrast_perimeter_files/FLIC_2001/walkIndoor/temporalFrequency/P_contrast-1x5_perimeter.mat
Model weights already exist!
Analyzing videos with /Users/zacharykelly/Library/Application Support/pylids/pytorch-pupil-v1/dlc-models-pytorch/iteration-0/santini_eyelid_detectionJul182025-trainset99shuffle1/train/snapshot-best-220.pt
Starting to analyze /Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2005/walkIndoor/temporalFrequency/P.avi
Video metadata: 
  Overall # of frames:    45833
  Duration of video [s]:  381.94
  fps:                    120.0
  resolution:             w=400, h=400

Running pose prediction with batch size 8


100%|██████████| 45833/45833 [17:38<00:00, 43.30it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpmgc0ordh/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpmgc0ordh/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpmgc0ordh/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2001/gazeCalibration/temporalFrequency has no contrast videos
['/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2003/walkIndoor/temporalFrequency/P_contrast-1x5.avi']
Generating: /Users/zacharykelly/Des

100%|██████████| 45833/45833 [17:26<00:00, 43.81it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2qsu0atv/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2qsu0atv/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp2qsu0atv/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2003/gazeCalibration/temporalFrequency has no contrast videos
['/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2004/walkIndoor/temporalFrequency/P_contrast-1x5.avi']
Generating: /Users/zacharykelly/Des

...

Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpfcbd4d3f/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpfcbd4d3f/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpfcbd4d3f/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2004/gazeCalibration/temporalFrequency has no contrast videos
['/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2005/walkIndoor/temporalFrequency/P_contrast-1x5.avi']
Generating: /Users/zacharykelly/Des

100%|██████████| 45833/45833 [17:29<00:00, 43.65it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr80danxj/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr80danxj/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmpr80danxj/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2005/gazeCalibration/temporalFrequency has no contrast videos
['/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2002/walkIndoor/temporalFrequency/P_contrast-1x5.avi']
Generating: /Users/zacharykelly/Des

100%|██████████| 45833/45833 [18:13<00:00, 41.91it/s]


Saving results in /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp0i703qrl/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5 and /var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp0i703qrl/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220_full.pickle
The videos are analyzed. Now your research can truly start!
You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.

['/var/folders/g_/1p95771n5l1f_f8gbbftkjy80000gn/T/tmp0i703qrl/PDLC_Resnet50_santini_eyelid_detectionJul182025shuffle1_snapshot_220.h5']
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2002/gazeCalibration/temporalFrequency has no contrast videos
/Volumes/T7 Shield/scriptedIndoorOutdoorVideos/FLIC_2006/walkIndoor/temporalFrequency has no contrast videos
/Volumes/T7 Shield/scriptedIndoorOu

In [ ]:
# Attempt to extract only eyelid features from this video 
eyelid_features: list[dict] = extract_eye_features.extract_pupil_features(path_to_video, 
                                                                          is_grayscale=True, 
                                                                          visualize_results=True, 
                                                                          method='pylids',
                                                                          safe_execution=True,
                                                                          keypoint_threshold=-1
                                                                         )

In [ ]:
scipy.io.savemat("/Users/zacharykelly/Desktop/FLIC_2005_walkIndoor_tf_perimeter.mat", {"perimeter": perimeter_dict})